<a href="https://colab.research.google.com/github/knuteriksen/VisionPorComputador/blob/main/imagecolor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Colorization

Thanks to [Emil Wallner](https://blog.floydhub.com/colorizing-b-w-photos-with-neural-networks/) for inspiration

# Imports


In [7]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.layers import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Initialization
Make sure that all keras stuff from previous sessions is cleared, and make sure we run in Eagerly mode. 

In [8]:
keras.backend.clear_session()
tf.config.run_functions_eagerly(True)

# Obtaining images

Get the training images from the local repository `Train/` and convert them to an array float type representation

In [9]:
# Get images
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

# Downloading the Inception ResNet V2 with imagenet weights

In [10]:
inception = InceptionResNetV2(weights='imagenet', include_top=True)

# Create the Encoder

Just a basic convoloutional encoder

In [11]:
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

# Create the fusion layer
The fusion layer takes as input both the output of the encoder and the output of the inception layer.

First Create an `Input` type. This variable will represent the output of the inception layer.

Then:

1.   Multiply the 1000 category Input layer `embed_input` (which contains the output of the inception layer) by 32*32 and store it in the `fusion_output` variable
2.   Then Reshape the `fusion_output` layer
3.   Then concatenate the fusion layer (now existing of the reshaped inception output) and the encoder output. **This is the actual fusion/concatination of the encoder and the inception.**
4. Then do a 1x1 convolution to obtain a final size of 256@32x32 that we need for the decoder.



In [12]:
embed_input = Input(shape=(1000,))
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = tf.keras.layers.Concatenate(axis=3)([encoder_output, fusion_output])
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

# Create the Decoder
The Decoder takes the output of the fusion layer as input

In [13]:
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

# Put together the whole model 
Remember that `embed_input` contains the output of the inception layer

In [14]:
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

# Get inception output
This output will be used as `embed_input` mentioned above.

1.    First resize the grayscaled input images to fit into the inception model
2.    Then use the preprocessor to feed the images in the correct format
3.    Then we run the input through inception resnet v2 network and extracts the final layer of the model.



In [15]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
      i = resize(i, (299, 299, 3), mode='constant')
      grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Data Generator
This is just  simple image data generator for training purposes

In [16]:
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

# Create Training Data
This is the actual generator the backend uses to obtain the correct training data.

1.   First we transform the rgb image to a grayscaled rgb
2.   Then we use the `create_inception_embedding` funtion to obtain the `embedding_input` metnioned above. Again, this corresponds to the output of the inception layer.
3.   Then we get the X and Y training data from the training dataset
4.   Then we yield the input corresponding to the model created above

In [17]:
batch_size = 20

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, embed], Y_batch)

# Compile and train model

In [18]:
tensorboard = TensorBoard(log_dir="/output")
model.compile(optimizer='adam', loss='mse')
model.fit(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=5)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
5/5 [==============================] - 82s 17s/step - loss: 0.2169
Epoch 2/10
5/5 [==============================] - 90s 19s/step - loss: 0.0070
Epoch 3/10
5/5 [==============================] - 90s 17s/step - loss: 0.0062
Epoch 4/10
5/5 [==============================] - 98s 21s/step - loss: 0.0058
Epoch 5/10
5/5 [==============================] - 90s 17s/step - loss: 0.0055
Epoch 6/10
5/5 [==============================] - 90s 17s/step - loss: 0.0060
Epoch 7/10
5/5 [==============================] - 90s 19s/step - loss: 0.0058
Epoch 8/10
5/5 [==============================] - 74s 15s/step - loss: 0.0054
Epoch 9/10
5/5 [==============================] - 82s 17s/step - loss: 0.0057
Epoch 10/10
5/5 [==============================] - 106s 21s/step - loss: 0.0053


# Make a prediction

In [22]:
color_me = []
for filename in os.listdir('Test/'):
    color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = 1.0/255*color_me
color_me = gray2rgb(rgb2gray(color_me))
color_me_embed = create_inception_embedding(color_me)
color_me = rgb2lab(color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


# Test the model

In [23]:
output = model.predict([color_me, color_me_embed])
output = output * 128

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


# Colorize the output of the test

In [24]:
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("Result/img_"+str(i)+".png", lab2rgb(cur))